## Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.mixed_precision import set_global_policy
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from glob import glob
import random
from PIL import Image
import seaborn as sns
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import mixed_precision

print("TensorFlow Version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
mixed_precision.set_global_policy("mixed_float16")  # Speed up training on GPU


# EDA

In [ ]:
import os
import pandas as pd

# Dataset directory path
directory_path = "/kaggle/input/ppd-v2/ppd_v1"

# Dictionary to store class names and image counts
class_counts = {}

# Iterate over each class folder
for class_name in os.listdir(directory_path):
    class_path = os.path.join(directory_path, class_name)
    if os.path.isdir(class_path):  # Ensure it's a folder
        image_count = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
        class_counts[class_name] = image_count

# Convert to DataFrame for better visualization
df = pd.DataFrame(list(class_counts.items()), columns=["Class", "Image Count"])

# Display results
print(df)

## Summary Statistics

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Display basic statistics
print(df.describe())


## Distribution of Image Counts

In [ ]:
# Set plot size
plt.figure(figsize=(14, 6))

# Bar plot of class distributions
sns.barplot(x="Class", y="Image Count", data=df, palette="viridis")

# Rotate x-axis labels for readability
plt.xticks(rotation=90)

# Labels and title
plt.xlabel("Class")
plt.ylabel("Number of Images")
plt.title("Image Distribution Across Classes")

# Show the plot
plt.show()


## Check All Image Extensions in the Dataset

In [ ]:
# Set to store unique file extensions
unique_extensions = set()

# Loop through each class folder
for pest_class in os.listdir(directory_path):
    class_dir = os.path.join(directory_path, pest_class)
    if os.path.isdir(class_dir):  # Ensure it's a directory
        for file in os.listdir(class_dir):
            ext = os.path.splitext(file)[1].lower()  # Extract file extension
            if ext:  # Ensure there's an extension
                unique_extensions.add(ext)

# Display all unique extensions found
print("📌 Unique file extensions in the dataset:", unique_extensions)

## Display 2 Images Per Class

In [ ]:
import os
import matplotlib.pyplot as plt
import random
import cv2
import numpy as np

# Dataset directory
directory_path = "/kaggle/input/ppd-v2/ppd_v1"

# Number of samples per class
samples_per_class = 2

# Get all class names
class_names = [name for name in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, name))]

# Define number of rows and columns
num_rows = len(class_names)
num_cols = samples_per_class

# Set figure size
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, num_rows * 2))

# Iterate over each class
for row_idx, class_name in enumerate(class_names):
    class_path = os.path.join(directory_path, class_name)
    
    # Get all images in the class directory
    image_files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
    
    # Select 2 random images
    selected_images = random.sample(image_files, min(samples_per_class, len(image_files)))
    
    # Display images
    for col_idx, image_file in enumerate(selected_images):
        img_path = os.path.join(class_path, image_file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        
        ax = axes[row_idx, col_idx]
        ax.imshow(img)
        ax.axis("off")
        ax.set_title(class_name if col_idx == 0 else "")

# Add separation lines
for row in range(num_rows - 1):  # Horizontal lines
    plt.axhline(y=row + 0.5, color='black', linewidth=2, linestyle="--")

for col in range(num_cols - 1):  # Vertical lines
    plt.axvline(x=col + 0.5, color='black', linewidth=2, linestyle="--")

plt.tight_layout()
plt.show()


## Overlooking on the Dataset

In [ ]:
pest_classes = os.listdir(directory_path)
print(f"📌 Found {len(pest_classes)} classes: {pest_classes}")


In [ ]:
VALID_EXTENSIONS = {".jpg", ".jpeg", ".png", ".webp", ".jfif", ".gif"}  # Valid image formats

# Dictionary to store image counts per class
image_counts = {}

# Loop through each class folder
for pest_class in os.listdir(directory_path):
    class_dir = os.path.join(directory_path, pest_class)
    if os.path.isdir(class_dir):  # Ensure it's a directory
        num_images = sum(
            len(glob(os.path.join(class_dir, f"*{ext}"))) for ext in VALID_EXTENSIONS
        )
        image_counts[pest_class] = num_images

# Convert to DataFrame for better visualization
df_counts = pd.DataFrame(image_counts.items(), columns=["Class", "Image Count"])

# Display total images in the dataset
total_images = df_counts["Image Count"].sum()
print(f"📌 Total images in dataset: {total_images}")
df_counts

# **DL model to do the automatic pest detection**

## Pest classification using an EfficientNetV2 model
Since each image contain one kind of pest, we will try using classification with EfficientNetV2

### Data preparation

In [ ]:
# Dataset Path
DATASET_PATH = "/kaggle/input/ppd-v2/ppd_v1"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data Augmentation
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Train & Validation Data
train_data = data_gen.flow_from_directory(
    DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', subset='training'
)
val_data = data_gen.flow_from_directory(
    DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', subset='validation'
)

### Model setup

In [ ]:
# Load Pretrained EfficientNetV2S
base_model = tf.keras.applications.EfficientNetV2S(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # Freeze initially

# Ensure BatchNorm Layers Stay Frozen
for layer in base_model.layers:
    if isinstance(layer, keras.layers.BatchNormalization):
        layer.trainable = False

# Classification Head
model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.4),  # Increased dropout
    keras.layers.Dense(train_data.num_classes, activation='softmax')
])

# Compile Model with Lower LR
model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4),  # Lowered LR
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


### Training

In [ ]:
# Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Training
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    steps_per_epoch=train_data.samples // BATCH_SIZE,
    validation_steps=val_data.samples // BATCH_SIZE,
    callbacks=[early_stopping, lr_scheduler]
)

# Unfreeze the Top Layers for Fine-Tuning
base_model.trainable = True
for layer in base_model.layers[:100]:  # Freeze bottom layers, unfreeze top layers
    layer.trainable = False

# Recompile Model with Lower LR
model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=1e-5),  # Even lower LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune Model
history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    steps_per_epoch=train_data.samples // BATCH_SIZE,
    validation_steps=val_data.samples // BATCH_SIZE,
    callbacks=[early_stopping, lr_scheduler]
)

### Evaluation

In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 4))
    
    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.legend()
    plt.title("Model Accuracy")
    
    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.legend()
    plt.title("Model Loss")
    
    plt.show()

plot_history(history)

### Model inference

In [ ]:
def predict_image(image_path, model):
    img = keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
    img_array = keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions[0])
    class_name = list(train_data.class_indices.keys())[class_idx]
    confidence = predictions[0][class_idx]
    
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Predicted: {class_name} ({confidence:.2f})")
    plt.show()

predict_image("/kaggle/input/ppd-v2/ppd_v1/Empoasca fabae/69483.jpg", model)

# Test 2

In [ ]:
# Import Data Science Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import itertools
import random

# Import visualization libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
import seaborn as sns

# Tensorflow Libraries
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model
from tensorflow.keras import preprocessing
# System libraries
from pathlib import Path
import os.path

# Metrics
from sklearn.metrics import classification_report, confusion_matrix

sns.set(style='darkgrid')

In [ ]:
def seed_everything(seed=42):
    # Seed value for TensorFlow
    tf.random.set_seed(seed)
    
    # Seed value for NumPy
    np.random.seed(seed)
    
    # Seed value for Python's random library
    random.seed(seed)
    
    # For TensorFlow 2.x, use these settings for deterministic behavior
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
    
    # Set global determinism for CUDA operations if using GPU
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    # Set thread determinism if you need multiprocessing
    # Note: This may impact performance
    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for our notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot

In [ ]:
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)

In [ ]:
# Walk through each directory
dataset = "/kaggle/input/ppd-v2/ppd_v1"
walk_through_dir(dataset)

In [ ]:
def convert_path_to_df(dataset):
    image_dir = Path(dataset)
    
    # List of all extensions to search for (in both lowercase and uppercase)
    extensions = ['.png', '.gif', '.jfif', '.jpg', '.webp', '.jpeg']
    extensions += [ext.upper() for ext in extensions]
    
    # Get filepaths for all specified extensions
    filepaths = []
    for ext in extensions:
        filepaths.extend(list(image_dir.glob(f'**/*{ext}')))
    
    # If no images found, return empty DataFrame
    if not filepaths:
        return pd.DataFrame(columns=['Filepath', 'Label'])
    
    # Extract labels from directory structure
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
    
    # Convert to Series
    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    
    # Concatenate filepaths and labels
    image_df = pd.concat([filepaths, labels], axis=1)
    
    return image_df

In [ ]:
# Check for corrupted images within the dataset
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

# List of all extensions to check
extensions = ['.png', '.gif', '.jfif', '.jpg', '.webp', '.jpeg']
extensions += [ext.upper() for ext in extensions]

corrupted_images = []
base_path = Path("/kaggle/input/ppd-v2/ppd_v1")

# Check each extension
for ext in extensions:
    path = base_path.rglob(f"*{ext}")
    for img_p in path:
        try:
            img = PIL.Image.open(img_p)
            # Optional: verify the image by loading it
            img.verify()
        except (PIL.UnidentifiedImageError, PIL.Image.DecompressionBombError, OSError, ValueError) as e:
            print(f"Corrupted image: {img_p} - Error: {e}")
            corrupted_images.append(img_p)

print(f"Total corrupted images found: {len(corrupted_images)}")

In [ ]:
label_counts = image_df['Label'].value_counts()

plt.figure(figsize=(10, 6))
sns.barplot(x=label_counts.index, y=label_counts.values, alpha=0.8, palette='rocket')
plt.title('Distribution of Labels in Image Dataset', fontsize=16)
plt.xlabel('Label', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(rotation=45) 
plt.show()


In [ ]:
len(image_df)

In [ ]:
# Filtrer le dataset en excluant certaines classes
excluded_labels = ['termite', 'fall armyworm', 'locust']
image_df = image_df[~image_df['Label'].isin(excluded_labels)].reset_index(drop=True)  # Réinitialiser l'index

# Générer de nouveaux indices aléatoires après filtrage
random_index = np.random.choice(len(image_df), 16, replace=False)

# Affichage des images
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(10, 10), subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.Filepath.iloc[random_index[i]]))  # Utiliser `.iloc` pour éviter les erreurs d'index
    ax.set_title(image_df.Label.iloc[random_index[i]])

plt.tight_layout()
plt.show()

In [ ]:
def compute_ela_cv(path, quality):
    """
    Compute Error Level Analysis using OpenCV
    
    Args:
        path: Path to the input image
        quality: JPEG compression quality (0-100)
    
    Returns:
        Difference image highlighting compression artifacts
    """
    temp_filename = 'temp_file_name.jpeg'
    SCALE = 15
    
    # Handle different file formats
    try:
        orig_img = cv2.imread(path)
        if orig_img is None:
            raise ValueError(f"Failed to load image: {path}")
        
        orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
        
        cv2.imwrite(temp_filename, orig_img, [cv2.IMWRITE_JPEG_QUALITY, quality])

        # Read compressed image
        compressed_img = cv2.imread(temp_filename)

        # Get absolute difference between img1 and img2 and multiply by scale
        diff = SCALE * cv2.absdiff(orig_img, compressed_img)
        
        # Clean up temp file
        if os.path.exists(temp_filename):
            os.remove(temp_filename)
            
        return diff
    
    except Exception as e:
        print(f"Error processing {path}: {e}")
        # Return a blank image of the same size
        if 'orig_img' in locals() and orig_img is not None:
            return np.zeros_like(orig_img)
        return None


def convert_to_ela_image(path, quality):
    """
    Convert image to Error Level Analysis image using PIL
    
    Args:
        path: Path to the input image
        quality: JPEG compression quality (0-100)
    
    Returns:
        PIL Image showing ELA result
    """
    temp_filename = 'temp_file_name.jpeg'
    
    try:
        # Open and convert to RGB (handles different formats)
        image = Image.open(path).convert('RGB')
        image.save(temp_filename, 'JPEG', quality=quality)
        temp_image = Image.open(temp_filename)

        ela_image = ImageChops.difference(image, temp_image)

        extrema = ela_image.getextrema()
        max_diff = max([ex[1] for ex in extrema])
        if max_diff == 0:
            max_diff = 1

        scale = 255.0 / max_diff
        ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
        
        # Clean up temp file
        if os.path.exists(temp_filename):
            os.remove(temp_filename)
            
        return ela_image
    
    except Exception as e:
        print(f"Error processing {path}: {e}")
        return None


def random_sample(path, extensions=None):
    """
    Get a random sample image from a directory
    
    Args:
        path: Directory path to search
        extensions: List of file extensions to include, or None for all files
    
    Returns:
        String path to a randomly selected file
    """
    if extensions is None:
        # Default to all your image formats
        extensions = ['png', 'gif', 'jfif', 'jpg', 'webp', 'jpeg', 
                      'PNG', 'GIF', 'JFIF', 'JPG', 'WEBP', 'JPEG']
    
    items = []
    
    # If extensions is a list, iterate through each extension
    if isinstance(extensions, list):
        for ext in extensions:
            items.extend(list(Path(path).glob(f'*.{ext}')))
    # If extensions is a single string, use it directly
    elif extensions:
        items = list(Path(path).glob(f'*.{extensions}'))
    # If no extensions specified, get all files
    else:
        items = list(Path(path).glob('*'))
    
    # Check if we found any files
    if not items:
        raise ValueError(f"No files found in {path} with extensions {extensions}")
        
    p = random.choice(items)
    return p.as_posix()

In [ ]:
# View random sample from the dataset
p = random_sample('/kaggle/input/ppd-v2/ppd_v1/Thrips tabaci')
orig = cv2.imread(p)
orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB) / 255.0
init_val = 100
columns = 3
rows = 3

fig=plt.figure(figsize=(15, 10))
for i in range(1, columns*rows +1):
    quality=init_val - (i-1) * 8
    img = compute_ela_cv(path=p, quality=quality)
    if i == 1:
        img = orig.copy()
    ax = fig.add_subplot(rows, columns, i) 
    ax.title.set_text(f'q: {quality}')
    plt.imshow(img)
plt.show()

In [ ]:
# Separate in train and test data
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input
)

In [ ]:
# Split the data into three categories.
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

In [ ]:
# Data Augmentation Step
augment = tf.keras.Sequential([
  tf.keras.layers.Resizing(224, 224),
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.RandomFlip("horizontal"),
  tf.keras.layers.RandomRotation(0.1),
  tf.keras.layers.RandomZoom(0.1),
  tf.keras.layers.RandomContrast(0.1),
])

In [ ]:
# Load the pretained model
pretrained_model = tf.keras.applications.efficientnet_v2.EfficientNetV2L(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

pretrained_model.trainable = False

In [ ]:
# Create checkpoint callback
checkpoint_path = "peanut_pests_classification_model_checkpoint.weights.h5"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                     save_weights_only=True,
                                     monitor="val_accuracy",
                                     save_best_only=True)

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

inputs = pretrained_model.input
x = augment(inputs)

x = Dense(128, activation='relu')(pretrained_model.output)
x = Dropout(0.45)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.45)(x)


# Change the final Dense layer to have 17 output units instead of 12
outputs = Dense(14, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=Adam(0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# history = model.fit(
#     train_images,
#     steps_per_epoch=len(train_images),
#     validation_data=val_images,
#     validation_steps=len(val_images),
#     epochs=100,
#     callbacks=[
#         early_stopping,
#         create_tensorboard_callback("training_logs", 
#                                     "peanut_pests_classification"),
#         checkpoint_callback,
#     ]
# )

history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=100,
    callbacks=[early_stopping, checkpoint_callback]  # Start with just one callback
)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Keep your base pretrained model and augmentation
inputs = pretrained_model.input
x = augment(inputs)

# Skip the GlobalAveragePooling2D since output is already 2D
# Start directly with the first dense layer
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(pretrained_model.output)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Second dense block with batch normalization and reduced dropout
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Output layer with 17 units (matching your target shape)
outputs = Dense(14, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Use a learning rate scheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Compile with a slightly higher initial learning rate
model.compile(
    optimizer=Adam(0.0001),  # Increased from 0.00001
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Add the learning rate scheduler to your callbacks
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=10,
    callbacks=[early_stopping, checkpoint_callback, reduce_lr]
)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Entrée depuis le modèle pré-entraîné
inputs = pretrained_model.input
x = augment(inputs)

# Vérifier si GlobalAveragePooling2D est nécessaire
if len(pretrained_model.output.shape) == 4:  # Vérifie si la sortie est en 4D
    x = GlobalAveragePooling2D()(pretrained_model.output)
else:
    x = pretrained_model.output  # Utiliser directement la sortie s'il est déjà en 2D

# Premier bloc dense avec plus de neurones
x = Dense(512, activation='relu', kernel_regularizer=l2(0.0005))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

# Deuxième bloc dense
x = Dense(256, activation='relu', kernel_regularizer=l2(0.0005))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Couche de sortie
outputs = Dense(14, activation='softmax')(x)

# Création du modèle
model = Model(inputs=inputs, outputs=outputs)

from tensorflow.keras.callbacks import LearningRateScheduler

# Définition du scheduler exponentiel
def lr_schedule(epoch, lr):
    return lr * 0.9 if epoch > 3 else lr  # Réduction progressive après 3 epochs

lr_scheduler = LearningRateScheduler(lr_schedule)

model.compile(
    optimizer=Adam(learning_rate=0.0003),  # Augmenté pour accélérer l'apprentissage
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=20,
    callbacks=[checkpoint_callback, lr_scheduler]
)
